In [30]:
import os
import numpy as np

import cantera as ct
import cantera.ck2cti
import rmgpy.chemkin

import subprocess

In [2]:
# load David's 2-BTP model
# https://github.com/comocheng/halogen_models/tree/main/combustion_symposium_paper/supporting_material

RMG_chemkin_path = 'models/RMG/chem_annotated.inp'
RMG_dictionary_path = 'models/RMG/species_dictionary.txt'
RMG_transport_path = 'models/RMG/tran.dat'
RMG_cti_path = 'models/RMG/chem_annotated.cti'

RMG_species_list, RMG_reaction_list = rmgpy.chemkin.load_chemkin_file(RMG_chemkin_path, dictionary_path=RMG_dictionary_path, transport_path=RMG_transport_path)
# RMG_gas = ct.Solution(RMG_cti_path)



In [3]:
# generate a list of species with halogens, then remove those from the model reactions
halogens = []
for sp in RMG_species_list:
    for atom in sp.molecule[0].atoms:
        if atom.is_halogen():
            halogens.append(sp)
            break
# generate a list of species with halogens, then remove those from the model reactions
halogen_reactions = []
for rxn in RMG_reaction_list:
    for reactant in rxn.reactants:
        if reactant in halogens:
            halogen_reactions.append(rxn)
            break

In [4]:
truncated_sp_list = []
for sp in RMG_species_list:
    if sp not in halogens:
        truncated_sp_list.append(sp)

truncated_rxn_list = []
for rxn in RMG_reaction_list:
    if rxn not in halogen_reactions:
        truncated_rxn_list.append(rxn)
        
print(len(truncated_sp_list))
print(len(truncated_rxn_list))


98
825


In [5]:
rmgpy.chemkin.save_chemkin_file('no_halogens.inp', truncated_sp_list, truncated_rxn_list, verbose=True, check_for_duplicates=True)
rmgpy.chemkin.save_transport_file('no_halogens_tran.dat', truncated_sp_list)
rmgpy.chemkin.save_species_dictionary('no_halogens_dict.txt', truncated_sp_list)

### Also create a truncated version of the NIST model

In [51]:
# Load NIST 2-BTP model
# /scratch/westgroup/David/halogen_models/models/2-BTP/NIST/2-BTP_kinetics.cti
NIST_cti_path = 'models/NIST/2-BTP_kinetics.cti'
NIST_dictionary_path = 'models/NIST/species_dictionary.txt'
NIST_chemkin_path = 'models/NIST/pym_gas_copy.inp'
NIST_transport_path = 'models/NIST/tran.dat'

NIST_species_list, NIST_reaction_list = rmgpy.chemkin.load_chemkin_file(NIST_chemkin_path, dictionary_path=NIST_dictionary_path, transport_path=NIST_transport_path)
NIST_gas = ct.Solution(NIST_cti_path)

print(len(NIST_species_list))
print(len(NIST_reaction_list))

For species CH2OCH, discontinuity in cp/R detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  8.393471510000001
	Value computed using high-temperature polynomial: 9.1801039121875

For species CH2OCH, discontinuity in h/RT detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  42.199147089791666
	Value computed using high-temperature polynomial: 41.961461604875005

For species CH2OCH, discontinuity in s/R detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  33.70692865946735
	Value computed using high-temperature polynomial: 33.51209988778391

For species C4H5-2, discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  47.65235236593109
	Value computed using high-temperature polynomial: 48.43623165666667

For species C4H5-2, discontinuity in s/R detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  52.42918829260522
	Value computed using high-temperature p

In [52]:
# generate a list of species with halogens, then remove those from the model reactions
halogens = []
for sp in NIST_species_list:
    for atom in sp.molecule[0].atoms:
        if atom.is_halogen():
            halogens.append(sp)
            break
# generate a list of species with halogens, then remove those from the model reactions
halogen_reactions = []
for rxn in NIST_reaction_list:
    for reactant in rxn.reactants:
        if reactant in halogens:
            halogen_reactions.append(rxn)
            break

In [53]:
truncated_sp_list = []
for sp in NIST_species_list:
    if sp not in halogens:
        truncated_sp_list.append(sp)

truncated_rxn_list = []
for rxn in NIST_reaction_list:
    if rxn not in halogen_reactions:
        truncated_rxn_list.append(rxn)
        
print(len(truncated_sp_list))
print(len(truncated_rxn_list))


116
808


In [55]:
model_dir = os.path.join('models', 'NIST_no_halogens')
os.makedirs(model_dir, exist_ok=True)

chemkin_file = os.path.join(model_dir, 'no_halogens.inp')
transport_file = os.path.join(model_dir, 'no_halogens_tran.dat')
species_dict = os.path.join(model_dir, 'no_halogens_dict.txt')


rmgpy.chemkin.save_chemkin_file(chemkin_file, truncated_sp_list, truncated_rxn_list, verbose=True, check_for_duplicates=True)
rmgpy.chemkin.save_transport_file(transport_file, truncated_sp_list)
rmgpy.chemkin.save_species_dictionary(species_dict, truncated_sp_list)

subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport_file}'])

Wrote CTI mechanism file to 'models/NIST_no_halogens/no_halogens.cti'.
Mechanism contains 116 species and 820 reactions.
Validating mechanism...CanteraWarning: NasaPoly2::validate: 
For species CH2OCH, discontinuity in cp/R detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  8.393471510000001
	Value computed using high-temperature polynomial: 9.1801039121875

For species CH2OCH, discontinuity in h/RT detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  42.199147089791666
	Value computed using high-temperature polynomial: 41.961461604875005

For species CH2OCH, discontinuity in s/R detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  33.70692865946735
	Value computed using high-temperature polynomial: 33.51209988778391

For species C4H5-2, discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  47.65235236593109
	Value computed using high-temperature polynomial: 48.4362316566

CompletedProcess(args=['ck2cti', '--input=models/NIST_no_halogens/no_halogens.inp', '--transport=models/NIST_no_halogens/no_halogens_tran.dat'], returncode=0)